In [1]:
#!pip install --quiet cornac==1.6.0 adjustText

  Retrying (Retry(total=4, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ConnectTimeoutError(<pip._vendor.urllib3.connection.VerifiedHTTPSConnection object at 0x7fc650bab6d0>, 'Connection to pypi.tuna.tsinghua.edu.cn timed out. (connect timeout=15)')': /simple/cornac/
^C
Operation cancelled by user


In [96]:
import os
import sys
from collections import defaultdict
import nltk

import string

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from adjustText import adjust_text
%matplotlib inline

import cornac
from cornac.utils import cache
from cornac.datasets import movielens
from cornac.eval_methods import RatioSplit
from cornac.models import MF, BPR, WMF
from cornac.data import Reader, SentimentModality

#%tensorflow_version 1.x
import tensorflow as tf

print(f"System version: {sys.version}")
print(f"Cornac version: {cornac.__version__}")
print(f"Tensorflow version: {tf.__version__}")

SEED = 42
VERBOSE = False

System version: 3.6.7 (default, Oct 25 2018, 09:16:13) 
[GCC 5.4.0 20160609]
Cornac version: 1.6.0
Tensorflow version: 1.11.0


In [ ]:
#read in review file
import pandas as pd
import gzip
import json

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df_review = getDF('Clothing_Shoes_and_Jewelry_5.json.gz')

In [6]:
#df_review.to_pickle('df_review.pickle')

In [3]:
df_review = pd.read_pickle('df_review.pickle')

In [9]:
df_review.shape

(11285464, 12)

In [24]:
df_review.head(10)

,rating,vote,verified,reviewTime,reviewerID,itemid,style,reviewerName,reviewText,summary,unixReviewTime,image
0,5.0,2,True,"05 4, 2014",A2IC3NZN488KWK,0871167042,{'Format:': ' Paperback'},Ruby Tulip,"This book has beautiful photos, good and under...",Unique designs,1399161600,NaN
1,5.0,NaN,True,"03 31, 2014",A30FG02C424EJ5,0871167042,{'Format:': ' Paperback'},NWCancerBaby,Loved their approach in this book and that it ...,Great Book,1396224000,NaN
2,5.0,NaN,True,"05 30, 2015",A2G9GWQEWWNQUB,0871167042,{'Format:': ' Paperback'},Pamelarenee,great,Five Stars,1432944000,NaN
3,5.0,NaN,True,"02 21, 2015",A3NI5OGW35SLY2,0871167042,{'Format:': ' Paperback'},Gail,"Always love the way Eva thinks, and there are ...",Great Book!,1424476800,NaN
4,5.0,NaN,True,"01 21, 2015",A1OPRA4NE56EV6,0871167042,{'Format:': ' Paperback'},carol a inman,Nice patterns,Five Stars,1421798400,NaN
5,4.0,NaN,True,"12 18, 2014",A3M6UXIK7XTA7A,0871167042,{'Format:': ' Paperback'},CV,love the spirit of the book and some of the pr...,Four Stars,1418860800,NaN
6,2.0,NaN,True,"09 3, 2014",A22ZX01TPWQY4G,0871167042,{'Format:': ' Paperback'},Nonnie,Not a fan. Limited designs. Limited content fo...,Two Stars,1409702400,NaN
7,3.0,5,True,"07 23, 2014",A1YIEW86G14BHP,0871167042,{'Format:': ' Paperback'},Aldne Now,"If you're an experienced wire jewelry maker, t...",OK Jewelry Book,1406073600,NaN
8,4.0,3,True,"06 5, 2014",AA7PNT2OPS3RP,0871167042,{'Format:': ' Paperback'},Catherine Franz,"Hi, have been making jewelry now for the last ...",Organic and upcycling jewelry maker,1401926400,NaN
9,5.0,2,True,"05 30, 2014",A3LOIIIW4G3TL7,0871167042,{'Format:': ' Paperback'},Picky Virgo,There's nothing prissy or matchy-matchy about ...,Perfect Title!,1401408000,NaN


In [22]:
df_review = df_review.rename(columns = {'asin':'itemid'})

In [23]:
df_review = df_review.rename(columns = {'overall':'rating'})

In [6]:
df_review.shape

(11285464, 12)

In [44]:
df_review_all = df_review

In [45]:
#remove na in review text
df_review_withimage = df_review_all.dropna()

In [46]:
df_review_withimage.columns

Index(['rating', 'vote', 'verified', 'reviewTime', 'reviewerID', 'itemid',
       'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime',
       'image'],
      dtype='object')

In [47]:
df_review_withtext = df_review_all.dropna(subset=['reviewText'])

In [48]:
df_review_withtext.shape

(11275566, 12)

In [49]:
#sample 1000000 rows for processing
df_review_withtext_sample = df_review_withtext.sample(n=1000000, random_state=1)

In [50]:
#df_review_withtext_sample.to_pickle('df_review_withtext_sample.pickle')
df_review_withtext_sample = pd.read_pickle('df_review_withtext_sample.pickle')

In [51]:

#convert tweets to lower case
df_review_withtext_sample['preprocess_data'] = df_review_withtext_sample['reviewText'].str.lower()


#removes extra spaces
df_review_withtext_sample['preprocess_data'] = df_review_withtext_sample['preprocess_data'].str.replace(r' +', ' ')

#punctuation
df_review_withtext_sample['preprocess_data'] = df_review_withtext_sample['preprocess_data'].str.replace('[^\w\s]','')

#split 
df_review_withtext_sample['preprocess_data'] = df_review_withtext_sample['preprocess_data'].str.lower().str.split()

In [52]:
df_review_withtext_sample['preprocess_data'].head()

9488852                                           [love, it]
7802048    [very, poorly, made, zipper, broke, almost, as...
6111777    [its, ok, i, returned, it, the, cross, body, s...
4670148    [these, fit, perfectly, i, have, issues, with,...
9872846    [bought, this, for, my, boyfriend, for, a, goo...
Name: preprocess_data, dtype: object

In [28]:
from nltk.corpus import stopwords
import nltk
import pickle
with open('stopwords.pickle', "rb") as f:
    stop = pickle.load(f)
 


In [29]:
stop.remove('and')

In [30]:
stop.remove('but')

In [53]:
#stop words removes
df_review_withtext_sample['preprocess_data'] = df_review_withtext_sample['preprocess_data'].apply(lambda x: [item for item in x if item not in stop])



In [56]:
#convert preprocessed list words to string 
df_review_withtext_sample['preprocess_str'] = df_review_withtext_sample['preprocess_data'].apply(lambda x: ' '.join(x) if x!=0 else x)
#df_review_withtext_sample['preprocess_str'] = df_review_withtext_sample['preprocess_data']

In [57]:
df_review_withtext_sample['preprocess_str']

9488852                                                  love
7802048     poorly made zipper broke almost soon started w...
6111777     ok returned cross body strap thin side think w...
4670148     fit perfectly issues almost tights every tight...
9872846     bought boyfriend good laugh but worn and says ...
5885524                                                    ok
7981666                                             described
2314583     reviews and even company said order 12 size im...
1824489                             like glasses like picture
9466244     advertised runs twice smaller size flimsy look...
10239463    love everything shirt except run small conside...
9007060                      fit well and look like made well
10978671                                          tall person
1837233                                             excellent
7258321     fit needs grandkids purchased several colors c...
8710504     love shoes but seem run little large ordered 8...
1104971 

In [59]:
#pip3 install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz
import en_core_web_sm

nlp = en_core_web_sm.load()

/usr/local/lib/python3.6/dist-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/usr/local/lib/python3.6/dist-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [22]:
#aspect terms extraction
def pos_tagging(data):
    print("pos tagging")
    req_tag = ['NN']
    extracted_words = []
    i = 0
    try:
        for x in data['preprocess_str']:
            doc = nlp(x)
            for token in doc:
                i += 1
                if token.tag_ in req_tag and token.shape_ != 'x' and token.shape_ != 'xx' and token.shape_ != 'xxx':
                    extracted_words.append(token.lemma_)
        return extracted_words
    except Exception as e:
        return extracted_words
        


In [ ]:
extract_words = pos_tagging(df_review_withtext_sample)

In [ ]:
extract_words

In [52]:
def word2vec(data):
    terms = listoflist(data)
    try:
        # removal of words which not present in the word2vec model vocabulary. (wrongly spelled)
        filtered_terms = []
        for i in range(len(terms)):
            corrent_words = [token for token in terms[i] if token in model_wiki.wv.vocab]
            if len(corrent_words) > 0 :
                filtered_terms.append(corrent_words[0])

        #converting words into vector
        vector_of_terms = []
        for x in range(len(filtered_terms)):
            vector_of_terms.append(model_wiki.wv[filtered_terms[x]])
        return vector_of_terms,filtered_terms
    except Exception as e:
        return abort(Response(
            json.dumps({'status_code': 400, 'success': False, 'message': 'Something went wrong'}),
            mimetype="application/json"))

In [ ]:
#pip3 install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz

In [60]:
#download from https://gist.github.com/mkulakowski2/4289437
text_file = open("negative-words.txt", "r")
neg = text_file.read().split('\n')

In [61]:
text_file = open("positive-words.txt", "r")
pos = text_file.read().split('\n')

In [62]:
# create a list of globally defined positive and negative words to identify sentiment
# sentiment score based on the laxicon neg, pos words

def feature_sentiment(sentence, pos, neg):
    '''
    input: dictionary and sentence
    function: appends dictionary with new features if the feature
              did not exist previously,then updates sentiment to
              each of the new or existing features
    output: updated dictionary
    '''
    sent_dict = dict()
    sent_tuple_lst = []
    sentence = nlp(sentence)
    opinion_words = neg + pos
    debug = 0
    for token in sentence:
        # check if the word is an opinion word, then assign sentiment
        if token.text in opinion_words:
            sentiment = 1 if token.text in pos else -1
            # if target is an adverb modifier (i.e. pretty, highly, etc.)
            # but happens to be an opinion word, ignore and pass
            if (token.dep_ == "advmod"):
                continue
            elif (token.dep_ == "amod"):
                #sent_dict[token.head.text] = sentiment
                sent_tuple_lst.append((token.head.text,token.text,sentiment))
            # for opinion words that are adjectives, adverbs, verbs...
            else:
                for child in token.children:
                    # if there's a adj modifier (i.e. very, pretty, etc.) add more weight to sentiment
                    # This could be better updated for modifiers that either positively or negatively emphasize
                    if ((child.dep_ == "amod") or (child.dep_ == "advmod")) and (child.text in opinion_words):
                        #sentiment *= 1.5
                        sentiment *= 1
                    # check for negation words and flip the sign of sentiment
                    if child.dep_ == "neg":
                        sentiment *= -1
                for child in token.children:
                    # if verb, check if there's a direct object
                    if (token.pos_ == "VERB") & (child.dep_ == "dobj"):                        
                        #sent_dict[child.text] = sentiment
                        sent_tuple_lst.append((child.text,token.text,sentiment))
                        # check for conjugates (a AND b), then add both to dictionary
                        subchildren = []
                        conj = 0
                        for subchild in child.children:
                            if subchild.text == "and":
                                conj=1
                            if (conj == 1) and (subchild.text != "and"):
                                subchildren.append(subchild.text)
                                conj = 0
                        for subchild in subchildren:
                            #sent_dict[subchild] = sentiment
                            sent_tuple_lst.append((subchild,token.text,sentiment))
                # check for negation
                for child in token.head.children:
                    noun = ""
                    if ((child.dep_ == "amod") or (child.dep_ == "advmod")) and (child.text in opinion_words):
                        #sentiment *= 1.5
                        sentiment *=1
                    # check for negation words and flip the sign of sentiment
                    if (child.dep_ == "neg"): 
                        sentiment *= -1
                
                # check for nouns
                for child in token.head.children:
                    noun = ""
                    if (child.pos_ == "NOUN") and (child.text not in sent_dict):
                        noun = child.text
                        # Check for compound nouns
                        for subchild in child.children:
                            if subchild.dep_ == "compound":
                                noun = subchild.text + " " + noun
                        #sent_dict[noun] = sentiment
                        sent_tuple_lst.append((noun,token.text,sentiment))
                    debug += 1
              
            
            
    #sent_tuple = [(k, v) for k, v in sent_dict.items()]        
    return sent_tuple_lst
    


In [63]:
# example 
sen = "wow super awesome product best highly recommended"
print (feature_sentiment(sen, pos, neg))


[('product', 'wow', 1), ('product', 'super', 1), ('product', 'awesome', 1), ('product', 'recommended', 1)]


In [64]:


def apply_extraction(row,nlp):
    review_body = row['preprocess_str']
    userid = row['reviewerID']
    itemid =row['itemid']

    doc=nlp(review_body)


    ## FIRST RULE OF DEPENDANCY PARSE -
    ## M - Sentiment modifier || A - Aspect
    ## RULE = M is child of A with a relationshio of amod
    rule1_pairs = []
    for token in doc:
        if token.dep_ == "amod":
            if token.text in pos:
                sentiment = 1
            elif token.text in neg:
                sentiment = -1
            else:
                sentiment =0
        
            rule1_pairs.append((token.head.text, token.text,sentiment))

    ## SECOND RULE OF DEPENDANCY PARSE -
    ## M - Sentiment modifier || A - Aspect
    #Direct Object - A is a child of something with relationship of nsubj, while
    # M is a child of the same something with relationship of dobj
    #Assumption - A verb will have only one NSUBJ and DOBJ

    rule2_pairs = []
    for token in doc:
        children = token.children
        A = "999999"
        M = "999999"
        for child in children :
            if(child.dep_ == "nsubj"):
                A = child.text
            if(child.dep_ == "dobj"):
                M = child.text
        if(A != "999999" and M != "999999"):
            
            if M in pos:
                sentiment = 1
            elif M in neg:
                sentiment = -1
            else:
                sentiment =0
            rule2_pairs.append((A, M,sentiment))

    ## THIRD RULE OF DEPENDANCY PARSE -
    ## M - Sentiment modifier || A - Aspect
    #Adjectival Complement - A is a child of something with relationship of nsubj, while
    # M is a child of the same something with relationship of acomp
    #Assumption - A verb will have only one NSUBJ and DOBJ

    rule3_pairs = []

    for token in doc:

        children = token.children
        A = "999999"
        M = "999999"
        for child in children :
            if(child.dep_ == "nsubj"):
                A = child.text

            if(child.dep_ == "acomp"):
                M = child.text

        if(A != "999999" and M != "999999"):
            
            if M in pos:
                sentiment = 1
            elif M in neg:
                sentiment = -1
            else:
                sentiment =0
            rule3_pairs.append((A, M,sentiment))

    ## FOURTH RULE OF DEPENDANCY PARSE -
    ## M - Sentiment modifier || A - Aspect

    #Adverbial modifier to a passive verb - A is a child of something with relationship of nsubjpass, while
    # M is a child of the same something with relationship of advmod

    #Assumption - A verb will have only one NSUBJ and DOBJ

    rule4_pairs = []
    for token in doc:


        children = token.children
        A = "999999"
        M = "999999"
        for child in children :
            if(child.dep_ == "nsubjpass"):
                A = child.text

            if(child.dep_ == "advmod"):
                M = child.text

        if(A != "999999" and M != "999999"):
            
            if M in pos:
                sentiment = 1
            elif M in neg:
                sentiment = -1
            else:
                sentiment =0
            rule4_pairs.append((A, M,sentiment))

    ## FIFTH RULE OF DEPENDANCY PARSE -
    ## M - Sentiment modifier || A - Aspect

    #Complement of a copular verb - A is a child of M with relationship of nsubj, while
    # M has a child with relationship of cop

    #Assumption - A verb will have only one NSUBJ and DOBJ

    rule5_pairs = []
    for token in doc:
        children = token.children
        A = "999999"
        buf_var = "999999"
        for child in children :
            if(child.dep_ == "nsubj"):
                A = child.text

            if(child.dep_ == "cop"):
                buf_var = child.text

        if(A != "999999" and buf_var != "999999"):
            
            if M in pos:
                sentiment = 1
            elif M in neg:
                sentiment = -1
            else:
                sentiment =0
            rule5_pairs.append((A, token.text,sentiment))
            
    aspects = []
    aspects = rule1_pairs + rule2_pairs + rule3_pairs +rule4_pairs +rule5_pairs
    #'userid','itemid',[('aspect','sentimentword','1or0')]
    result = (userid ,itemid, aspects)
    return result

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [186]:
df_review_withtext_sample = df_review_withtext_sample.reset_index(drop=True)

In [65]:
df_review_withtext_sample['sentiment']=''

In [66]:
%%time
df_review_withtext_sample['sentiment'] = df_review_withtext_sample.apply(lambda row: apply_extraction(row,nlp), axis=1)

#df_review_withtext_sample.to_pickle('processed_with_sentiment.pickle')

CPU times: user 2h 41min 22s, sys: 8.57 s, total: 2h 41min 31s
Wall time: 2h 41min 39s


In [97]:
df_review_withtext_sample = pd.read_pickle('processed_with_sentiment.pickle')

In [98]:
df_review_withtext_sample['sentiment'].tail()

11040177    (A3D0TUJC497YBC, B0176UF0W4, [(little, nice, 1)])
10993076    (A1TK3ZYY13QSNI, B015F1HS8A, [(product, super,...
7402889     (AP3NJCBFQ3CYT, B00W8BB9EA, [(woman, super, 1)...
3778676     (AFJIKJ97WVNCU, B009GPPMWG, [(thigh, perfect, ...
5388130     (A3F3U8TTMBAW4Z, B00I4VPF84, [(shoes, great, 1...
Name: sentiment, dtype: object

In [99]:
df_review_withtext_sample.columns

Index(['rating', 'vote', 'verified', 'reviewTime', 'reviewerID', 'itemid',
       'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime',
       'image', 'preprocess_data', 'preprocess_str', 'sentiment'],
      dtype='object')

In [100]:
df_new = df_review_withtext_sample[['reviewerID','itemid','rating','sentiment']]

In [101]:
df_new.columns = ['user','item','rating','aspect-level sentiment']

In [102]:
# Use Sentiment Modality for aspect-level sentiment data
sentiment_list = df_new['aspect-level sentiment'].tolist()
sentiment_modality = SentimentModality(data=sentiment_list)

# construct rating as a list of tuples (user, item, rating)
rating = list(df_new[['user','item','rating']].itertuples(index=False,name = None))


rs = RatioSplit(
    data=rating,
    test_size=0.2,
    exclude_unknowns=True,
    sentiment=sentiment_modality,
    verbose=VERBOSE,
    seed=SEED,
)
print("Total number of aspects:", rs.sentiment.num_aspects)
print("Total number of opinions:", rs.sentiment.num_opinions)


/usr/local/lib/python3.6/dist-packages/cornac/data/dataset.py:361: UserWarning: 1978 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)
/usr/local/lib/python3.6/dist-packages/cornac/data/dataset.py:361: UserWarning: 74 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


Total number of aspects: 48581
Total number of opinions: 43225


In [105]:
df_new['aspect-level sentiment'].tail()


11040177    (A3D0TUJC497YBC, B0176UF0W4, [(little, nice, 1)])
10993076    (A1TK3ZYY13QSNI, B015F1HS8A, [(product, super,...
7402889     (AP3NJCBFQ3CYT, B00W8BB9EA, [(woman, super, 1)...
3778676     (AFJIKJ97WVNCU, B009GPPMWG, [(thigh, perfect, ...
5388130     (A3F3U8TTMBAW4Z, B00I4VPF84, [(shoes, great, 1...
Name: aspect-level sentiment, dtype: object

In [107]:
from cornac.models import EFM, MTER, NMF, BPR
VERBOSE = True
efm = EFM(
  num_explicit_factors=20,
  num_latent_factors=30,
  num_most_cared_aspects=10,
  rating_scale=5.0,
  alpha=0.85,
  lambda_x=1,
  lambda_y=1,
  lambda_u=0.01,
  lambda_h=0.01,
  lambda_v=0.01,
  max_iter=100,
  verbose=VERBOSE,
  seed=SEED,
  name=f"EFM",
)

eval_metrics = [
    cornac.metrics.AUC(),
    cornac.metrics.FMeasure(k=50),
    cornac.metrics.Recall(k=50),
    cornac.metrics.NDCG(k=50),
    cornac.metrics.MRR(),
    cornac.metrics.Precision(k=50),
]

cornac.Experiment(
  eval_method=rs, models=[efm], metrics=eval_metrics
).run()


Building matrices completed!
iter: 1, loss: 38288860.000000
iter: 2, loss: 3221199.250000
iter: 3, loss: 2004746.375000
iter: 4, loss: 1627539.750000
iter: 5, loss: 1443958.875000
iter: 6, loss: 1333440.375000
iter: 7, loss: 1256828.875000
iter: 8, loss: 1198523.000000
iter: 9, loss: 1151384.750000
iter: 10, loss: 1111627.875000
iter: 11, loss: 1077114.000000
iter: 12, loss: 1046738.187500
iter: 13, loss: 1018803.875000
iter: 14, loss: 993380.312500
iter: 15, loss: 970017.937500
iter: 16, loss: 948325.375000
iter: 17, loss: 928149.312500
iter: 18, loss: 909271.000000
iter: 19, loss: 891541.562500
iter: 20, loss: 874834.187500
iter: 21, loss: 858993.062500
iter: 22, loss: 844008.625000
iter: 23, loss: 829755.625000
iter: 24, loss: 816241.187500
iter: 25, loss: 803311.750000
iter: 26, loss: 791002.000000
iter: 27, loss: 779228.687500
iter: 28, loss: 767893.625000
iter: 29, loss: 757039.375000
iter: 30, loss: 746597.875000
iter: 31, loss: 736571.562500
iter: 32, loss: 726901.312500
iter: 

In [77]:
id_aspect_map = {v:k for k, v in rs.sentiment.aspect_id_map.items()}
id_opinion_map = {v:k for k, v in rs.sentiment.opinion_id_map.items()}

In [78]:
efm = EFM()
efm.train_set = rs.train_set
_, X, Y = efm._build_matrices(rs.train_set)


In [81]:
n_users = 12
n_aspects =10
pd.DataFrame(
  data=X[:n_users, :n_aspects].A,
  index=[f"User {u + 1}" for u in np.arange(n_users)],
  columns=[f"{id_aspect_map[i]}" for i in np.arange(n_aspects)]
)

,buttons,jacket,think,side,purse,bag,reviews,bags,quality,luggage
User 1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
User 2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
User 3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.620593,0.0
User 4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
User 5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
User 6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.848469,0.0
User 7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
User 8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
User 9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
User 10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [82]:

n_items = 5
n_aspects = 10
pd.DataFrame(
  data=Y[:n_items, :n_aspects].A,
  index=[f"Item {u + 1}" for u in np.arange(n_items)],
  columns=[f"{id_aspect_map[i]}" for i in np.arange(n_aspects)]
)

,buttons,jacket,think,side,purse,bag,reviews,bags,quality,luggage
Item 1,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
Item 2,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
Item 3,0.0,0.0,0.000000,0.000000,0.0,0.0,3.0,0.0,4.928055,0.0
Item 4,0.0,0.0,3.924234,3.000000,0.0,0.0,3.0,0.0,4.523188,0.0
Item 5,0.0,0.0,0.000000,2.075766,0.0,0.0,3.0,0.0,4.810297,0.0


In [83]:
import pickle
with open('efm.pickle', 'wb') as handle:
    pickle.dump(efm, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('efm.pickle', 'rb') as handle:
    b = pickle.load(handle)